<a href="https://colab.research.google.com/github/RamyHamrouni/NLP-notebooks/blob/main/Presentation_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import logging
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%pip install langchain_tavily

# Task
Create a LangGraph orchestrator pattern with dummy tools and run it.

## Install langgraph

### Subtask:
Install the necessary library.


**Reasoning**:
The subtask is to install the `langgraph` library. This can be done using the `pip install` command in a code cell.



In [3]:
%pip install -U langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 12.9 MB/s eta 0:00:00


In [111]:
import logging
import numpy as np
import matplotlib.pyplot as plt
from openai import OpenAI
from typing import List, Dict, Any, Optional, Type, TypeVar
from pydantic import BaseModel
import json

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

T = TypeVar("T", bound=BaseModel)

class LLMClient:
    def __init__(self, base_url: str, api_key: str):
        self.base_url = base_url
        self.api_key = api_key
        logger.info(f"Initialized LLMClient with base_url: {self.base_url}")

    def completion(self, user_input: str, tools: List[Dict[str, Any]] = None) -> Any:
        raise NotImplementedError("completion must be implemented by subclasses")


class OpenAIClient(LLMClient):
    def __init__(self, base_url: str, api_key: str, model: str = "gpt-4.1-mini", temperature: float = 0.2):
        super().__init__(base_url, api_key)
        self.model = model
        self.client = OpenAI(api_key=api_key, base_url=base_url)
        self.temperature = temperature
        logger.info(f"Initialized OpenAIClient with model: {self.model}, temperature: {self.temperature}")


    def completion(
        self,
        messages: List[Dict[str, str]],
        tools: Optional[List[Dict[str, Any]]] = None,
    ) -> Any:
        params = {
            "model": self.model,
            "messages": messages,
            "temperature": self.temperature,
        }
        logger.info(f"Calling OpenAI completion with params: {params}")

        if tools is not None:
            params["tools"] = tools
            params["tool_choice"] = "auto"
            logger.info(f"Adding tools to completion params: {tools}")


        response = self.client.chat.completions.create(**params)
        logger.info("Received response from OpenAI completion.")
        return response.choices[0].message

    # 🧠 Structured output method — like LangChain’s with_structured_output
    def structured_completion(
        self,
        messages: List[Dict[str, str]],
        schema: Type[T],
        tools: Optional[List[Dict[str, Any]]] = None
    ) -> T:
        """
        Generate structured output that conforms to a given Pydantic schema.
        """
        logger.info("Calling OpenAI structured completion.")

        params = {
            "model": self.model,
            "messages": messages,
            "temperature": self.temperature,
        }
        if tools is not None:
            params["tools"] = tools
            params["tool_choice"] = "auto"
            logger.info(f"Adding tools to structured completion params: {tools}")
        logger.debug(schema)
        #params["response_format"] = schema



        logger.info(f"Structured completion params: {params}")


        response = self.client.chat.completions.create(**params)
        logger.info("Received response from OpenAI structured completion.")


        content = response.choices[0].message.content
        logger.info(f"Raw structured output: {content}")
        try:
            data = json.loads(content)
            logger.info("Successfully parsed structured output.")
            return schema.model_validate(data)
        except Exception as e:
            logger.error(f"Failed to parse structured output: {e}\nRaw output: {content}")
            raise ValueError(f"Failed to parse structured output: {e}\nRaw output: {content}")



iteration : 4
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_tnZ1XlzQPCQtfSwPbB1wuKHL', function=Function(arguments='{"query":"future of generative AI: trends, challenges, opportunities, ethical concerns, industry impact, technological advancements, regulations, and societal implications","max_results":6}', name='tavily_search'), type='function', index=0)], reasoning=None)
iteration : 3
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_dKoraSYGEOjmIcXbAaBQNXUJ', function=Function(arguments='{"query":"future of generative AI: key trends, challenges, opportunities, ethical issues, industry impact, regulations, and technological advancements","max_results":5}', name='tavily_search'), type='function', index=0)], reasoning=None)


In [131]:
def strategist_agent(startegist_context_manager,tools,CreativeBrief,tools_executor) -> CreativeBrief:
    # Step 1: LLM completes the task, calling web_search internally if needed
    llm_client=OpenAIClient(
          api_key="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",
          base_url="https://openrouter.ai/api/v1",
          model="gpt-5-mini",
          temperature=0.5
    )

    response = llm_client.completion(
        messages=startegist_context_manager.messages,
        tools=tools
    )
    max_iter = 4

    # Step 2: Handle multiple tool calls iteratively
    while response.tool_calls and max_iter > 0:
        print("iteration :",max_iter)
        print(response)
        startegist_context_manager.add_message(role="assistant", content=response.tool_calls)
        max_iter -= 1
        for tool_call in response.tool_calls:
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)

            # Execute the web search
            if function_name == "tavily_search":
                search_results = tools_executor["web_search"](**arguments)

                # Aggregate results: attach summaries, titles, URLs
                summarized_results = [
                    f"{r['title']}: {r['content']} ({r['url']})"
                    for r in search_results['results']
                ]
                startegist_context_manager.add_message(
                    role="assistant",
                    content=f"Tool called: {json.dumps(summarized_results)}"
                )

        # Step 3: Send updated messages back to LLM
        response = llm_client.completion(
            messages=startegist_context_manager.messages+[{"role": "system", "content": "Check if all aspects of the topic are covered. If not, perform additional searches and provide new sources. If everything is already covered, confirm coverage."}],
            tools=tools
        )
        print(response)


    # Step 4: Once all tool calls are resolved, generate structured CreativeBrief
    final_brief = llm_client.structured_completion(
        messages=startegist_context_manager.messages,
        schema=CreativeBrief
    )
    startegist_context_manager.add_message(role="assistant", content=final_brief)

    return final_brief




In [118]:
class ContextManager:
    def __init__(self,messages:List):
        self.messages = messages

    def add_message(self, role: str, content: str):
        self.messages.append({"role": role, "content": content})

In [119]:
def create_expanded_context(
        base_prompt: str,
        role: Optional[str] = None,
        examples: Optional[List[str]] = None,
        constraints: Optional[List[str]] = None,
        audience: Optional[str] = None,
        tone: Optional[str] = None,
        output_format: Optional[str] = None
    ) -> str:
          """
          Create an expanded context from a base prompt with optional components.

          Args:
              base_prompt: The core instruction or question
              role: Who the model should act as
              examples: List of example outputs to guide the model
              constraints: List of requirements or boundaries
              audience: Who the output is intended for
              tone: Desired tone of the response
              output_format: Specific format requirements

          Returns:
              Expanded context as a string
          """
          context_parts = []

          # Add role if provided
          if role:
              context_parts.append(f"You are {role}.")

          # Add base prompt
          context_parts.append(base_prompt)

          # Add audience if provided
          if audience:
              context_parts.append(f"Your response should be suitable for {audience}.")

          # Add tone if provided
          if tone:
              context_parts.append(f"Use a {tone} tone in your response.")

          # Add output format if provided
          if output_format:
              context_parts.append(f"Format your response as {output_format}.")

          # Add constraints if provided
          if constraints and len(constraints) > 0:
              context_parts.append("Requirements:")
              for constraint in constraints:
                  context_parts.append(f"- {constraint}")

          # Add examples if provided
          if examples and len(examples) > 0:
              context_parts.append("Examples:")
              for i, example in enumerate(examples, 1):
                  context_parts.append(f"Example {i}:\n{example}")

          # Join all parts with appropriate spacing
          expanded_context = "\n\n".join(context_parts)

          return expanded_context

In [120]:
import os
from google.colab import userdata

# Get Tavily API key
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

# Get OpenAI API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [121]:
def tavily_search(query: str, max_results: int = 3):
    search = TavilySearch(max_results=max_results)
    results = search.invoke(query)
    return results
tools = [
    {
        "type": "function",
        "function": {
            "name": "tavily_search",
            "description": "Searches the web using Tavily and returns top results for a query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query to search for on the web."
                    },
                    "max_results": {
                        "type": "integer",
                        "description": "Maximum number of search results to return.",
                        "default": 3
                    }
                },
                "required": ["query"]
            }
        }
    }
]

In [122]:
from pydantic import BaseModel, Field
class CreativeBrief(BaseModel):
      target_audience: str | list = Field(description="The target audience for the brief")
      primary_goal: str = Field(description="The primary goal of the brief")
      professional_tone: str = Field(description="The professional tone of the brief")
      key_takeaway: str = Field(description="The key takeaway from the brief")
      research_context: dict = Field(description="The research context for the brief")


In [132]:


STRATEGIST_SYSTEM_PROMPT = create_expanded_context(
    base_prompt="""
    Your job is to take a user's topic and create a 'Creative Brief' for the presentation team.

    You must define:
      - target_audience
      - primary_goal (e.g., 'to inform', 'to persuade', 'to get funding')
      - professional_tone (e.g., 'casual', 'formal', 'academic')
      - key_takeaway
      - research context (deep research gathered from multiple web searches to cover all aspects)
    """,
    role="You are an expert communications strategist and audience analyst.",
    output_format="JSON",
    constraints="If the user's input lacks important details, perform the necessary web searches to gather basic context before generating the brief."
)



strategist_messages = [
    {"role": "system", "content": STRATEGIST_SYSTEM_PROMPT},
    {"role": "user", "content": "Create a presentation about the futur of generative ai, all aspects"}
]
tools_executor={
    "web_search": tavily_search
}
startegist_context_manager = ContextManager(strategist_messages)
strategist_agent=strategist_agent(startegist_context_manager,tools,CreativeBrief=CreativeBrief,tools_executor=tools_executor)

iteration : 4
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_rgiIYSfHo4IWOQqMLtzuK1Is', function=Function(arguments='{"query":"future of generative AI trends 2025 2030 research reports articles","max_results":5}', name='tavily_search'), type='function', index=0), ChatCompletionMessageFunctionToolCall(id='call_pSxjUej0uQ5yS4SjlzTEMrgE', function=Function(arguments='{"query":"applications of generative AI across industries examples healthcare finance art education manufacturing 2024 2025","max_results":5}', name='tavily_search'), type='function', index=1), ChatCompletionMessageFunctionToolCall(id='call_8QIWY4EQZHzM6UgMkqeHgrCk', function=Function(arguments='{"query":"risks ethics regulation generative AI deepfakes bias copyright 2024 2025 policy","max_results":5}', name='tavily_search'), type='function', index=2), ChatCompletionMessageFunctionToolCall(id='call_K

In [133]:
startegist_context_manager.messages[-1]

{'role': 'assistant',
 'content': CreativeBrief(target_audience=['C-suite executives (CEOs, CTOs, CIOs) evaluating strategic investment in AI', 'Product and engineering leaders planning GenAI adoption', 'Policy makers and compliance/legal teams responsible for AI governance', 'Investors and business development leads assessing market opportunities', 'HR and learning & development leads planning workforce reskilling', 'Practitioners and technical leads who will implement solutions'], primary_goal='To inform stakeholders about the full landscape and trajectory of generative AI (technology, applications, economics, regulation, risks, and infrastructure) and to persuade decision-makers to adopt a balanced strategy that invests in capability, governance, sustainability, and workforce reskilling.', professional_tone='Business-formal: authoritative and strategic but accessible to both technical and non-technical stakeholders.', key_takeaway='Generative AI is rapidly evolving into multimodal, 

In [75]:
# Define the base system prompt
STRATEGIST_SYSTEM_PROMPT = create_expanded_context(
    base_prompt="""
    Your job is to take a user's topic and create a 'Creative Brief' for the presentation team.

    You must define:
      - target_audience
      - primary_goal (e.g., 'to inform', 'to persuade', 'to get funding')
      - professional_tone (e.g., 'casual', 'formal', 'academic')
      - key_takeaway
      - research context
    """,
    role="You are an expert communications strategist and audience analyst.",
    output_format="JSON",
    constraints="If the user's input lacks important details, perform the necessary web searches to gather basic context before generating the brief."
)



strategist_messages = [
    {"role": "system", "content": STRATEGIST_SYSTEM_PROMPT},
    {"role": "user", "content": "Create a presentation about the futur of generative ai, all aspects"}
]
tools_executor={
    "web_search_tool": web_search
}
startegist_context_manager = ContextManager(strategist_messages)
strategist_agent=strategist_agent("Create a presentation about the futur of generative ai, all aspects",startegist_context_manager,tools,tools_executor)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_aXT8vtUH53fbSfoiuwzOnzuZ', function=Function(arguments='{"query":"future of generative AI","max_results":3}', name='tavily_search'), type='function', index=0)], reasoning=None), native_finish_reason='completed')


AttributeError: 'Choice' object has no attribute 'tool_calls'

In [21]:

from langchain.agents import initialize_agent
tools = [web_search]

agent = initialize_agent(
    tools,
    llm_client,
    agent_type="chat-zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

# 5️⃣ Run the Strategist Agent
topic = "Create a presentation about the future of generative AI, all aspects."
response = agent.run(f"{STRATEGIST_SYSTEM_PROMPT}\n\nTopic: {topic}")

print("\n🎯 Structured Creative Brief:")
print(response)

/tmp/ipython-input-2986039617.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


ValueError: ZeroShotAgent does not support multi-input tool tavily_search.

In [20]:
startegist_context_manager.messages

[{'role': 'system',
  'content': "You are You are an expert communications strategist and audience analyst..\n\n\n    Your job is to take a user's topic and create a 'Creative Brief' for the rest of the team.\n    You must define:\n      - target_audience\n      - primary_goal (e.g., 'to inform', 'to persuade', 'to get funding')\n      - professional_tone (e.g., 'casual', 'formal', 'academic')\n      - key_takeaway\n\n    Return your response as a structured JSON object.\n    "},
 {'role': 'user',
  'content': 'Create a presentation about the futur of generative ai, all aspects'},
 {'role': 'assistant',
  'content': '{"target_audience":"Tech professionals, AI researchers, business leaders, and innovation enthusiasts interested in the future developments and implications of generative AI.","primary_goal":"to inform and inspire the audience about the comprehensive future prospects, challenges, and opportunities of generative AI across various sectors.","professional_tone":"formal with an

In [18]:
RESEARCHER_SYSTEM_PROMPT = create_expanded_context(
    base_prompt="""
   Using the 'Creative Brief' from the Strategist, your job is to find the most relevant facts, statistics, case studies, and supporting data.
   You must focus only on information that supports the goal and key takeaway for the target audience.
    """,
    role="You are a diligent research analyst. ",
)
researcher_messages = [
    {"role": "system", "content": RESEARCHER_SYSTEM_PROMPT},
    {"role": "user", "content": "Create a presentation about the futur of generative ai, all aspects"}
]
startegist_context_manager = ContextManager(strategist_messages)

brief = llm_client.structured_completion(messages, CreativeBrief)
startegist_context_manager.add_message("assistant", brief.model_dump_json())
print("🎯 Structured Creative Brief:")
print(brief)
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/react")
tools =  [
    web_search
]

agent = create_react_agent(model, tools, RESEARCHER_SYSTEM_PROMPT)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": "hi"})

# Use with chat history
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name?",
        # Notice that chat_history is a string
        # since this prompt is aimed at LLMs, not chat models
        "chat_history": "Human: My name is Bob\nAI: Hello Bob!",
    }
)

{'target_audience': 'Tech professionals, AI researchers, business leaders, and innovation enthusiasts interested in the advancements and implications of generative AI.',
 'primary_goal': 'to inform and inspire the audience about the future developments, applications, challenges, and ethical considerations of generative AI across various sectors.',
 'professional_tone': 'formal yet engaging, balancing technical depth with accessibility to appeal to a diverse professional audience.',
 'key_takeaway': 'Generative AI is poised to revolutionize multiple industries by enabling unprecedented creativity and efficiency, but its future success depends on addressing ethical challenges, ensuring responsible use, and fostering interdisciplinary collaboration.'}

In [ ]:
research_agent=

In [ ]:
from langgraph.graph import StateGraph, END

# Define the state

class GraphState:
    def __init__(self):
        self.ContextManager = ContextManager(messages=[])
        self.ContextManager.add_message("system", "You are a helpful assistant.")
        self.LLMClient= OpenAIClient(
          api_key="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",
          base_url="https://openrouter.ai/api/v1",
          model="gpt-5-mini"
        )
        self.next = None

# Create an instance of StateGraph
graph = StateGraph(GraphState)

# Define a node to handle tool calls
def call_tool(state: GraphState):
    # In a real scenario, this node would execute the tool calls
    # and update the state with the results.
    print("Calling tool...")
    state.messages.append("Tool called")
    state.next = "end"  # Simple flow: tool calls lead to the end
    return state

# Add the defined nodes to the StateGraph instance
graph.add_node("orchestrator", orchastrato)

# Set the entry point of the graph
graph.set_entry_point("tool_caller")

# Set the finish point of the graph
graph.set_finish_point("tool_caller") # For this simple example, the tool caller leads to the end

# Compile the graph
app = graph.compile()

## Define tools

### Subtask:
Define the tools that the orchestrator will use.


**Reasoning**:
Import the necessary decorator and define the dummy tools as instructed.



In [ ]:
from langchain_core.tools import tool

@tool
def tool1(query: str) -> str:
    """Calls tool1 with the given query."""
    return f"Tool1 was called with: {query}"

@tool
def tool2(query: str) -> str:
    """Calls tool2 with the given query."""
    return f"Tool2 was called with: {query}"

tools = [tool1, tool2]

## Define the graph

### Subtask:
Define the LangGraph graph with nodes and edges representing the orchestrator pattern.


**Reasoning**:
Import necessary classes, define the state, create the graph instance, define and add nodes, set the entry and finish points, and compile the graph.



## Run the graph

### Subtask:
Run the defined graph to demonstrate the orchestrator pattern.


**Reasoning**:
Define the initial state and invoke the compiled graph with this state. Then print the result.



In [ ]:
# Define the initial state
initial_state = GraphState()

# Invoke the compiled graph
result = app.invoke(initial_state)

# Print the output
print("Final State:", result)

InvalidUpdateError: Expected dict, got <__main__.GraphState object at 0x7cd245aca5a0>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

**Reasoning**:
The error message indicates that the node `call_tool` is returning a `GraphState` object, but it should be returning a dictionary representing the state updates. Modify the `call_tool` function to return a dictionary.



In [ ]:
# Define a node to handle tool calls
def call_tool(state: GraphState):
    # In a real scenario, this node would execute the tool calls
    # and update the state with the results.
    print("Calling tool...")
    # Return a dictionary with the updates to the state
    return {"messages": state.messages + ["Tool called"], "next": "end"}

# Re-compile the graph
app = graph.compile()

# Define the initial state
initial_state = GraphState()

# Invoke the compiled graph
result = app.invoke(initial_state)

# Print the output
print("Final State:", result)

InvalidUpdateError: Expected dict, got <__main__.GraphState object at 0x7cd245b96240>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

**Reasoning**:
The error persists because the graph is still configured to return the entire `GraphState` object when setting the finish point. The finish point should represent the final state or output, which in this case should be a dictionary reflecting the state after the tool call. Modify the graph definition to explicitly indicate that the output is the state dictionary when setting the finish point.



In [ ]:
from langgraph.graph import StateGraph, END

# Define the state - already defined in a previous cell
# class GraphState:
#     def __init__(self):
#         self.messages = []
#         self.tool_calls = None
#         self.next = None

# Create an instance of StateGraph - already created
# graph = StateGraph(GraphState)

# Define a node to handle tool calls - already defined and modified
def call_tool(state: GraphState):
    print("Calling tool...")
    return {"messages": state.messages + ["Tool called"], "next": "end"}

# Add the defined nodes to the StateGraph instance - already done
# graph.add_node("tool_caller", call_tool)

# Set the entry point of the graph - already done
# graph.set_entry_point("tool_caller")

# Set the finish point of the graph to return the state dictionary
graph.set_finish_point("tool_caller", mapper=lambda x: x.__dict__)

# Compile the graph
app = graph.compile()

# Define the initial state - already done
initial_state = GraphState()

# Invoke the compiled graph
result = app.invoke(initial_state)

# Print the output
print("Final State:", result)

TypeError: StateGraph.set_finish_point() got an unexpected keyword argument 'mapper'

**Reasoning**:
The error indicates that `set_finish_point` does not accept a `mapper` argument. Looking at the LangGraph documentation, the finish point simply indicates the node where the graph execution stops. The final output is typically the state of the graph at that point. The previous error was likely due to the node itself returning an object instead of a dictionary. Since the `call_tool` node is now returning a dictionary, setting the finish point to this node should work correctly.



In [ ]:
from langgraph.graph import StateGraph, END

# Define the state - already defined in a previous cell
# class GraphState:
#     def __init__(self):
#         self.messages = []
#         self.tool_calls = None
#         self.next = None

# Create an instance of StateGraph - already created
# graph = StateGraph(GraphState)

# Define a node to handle tool calls - already defined and modified
def call_tool(state: GraphState):
    print("Calling tool...")
    return {"messages": state.messages + ["Tool called"], "next": "end"}

# Add the defined nodes to the StateGraph instance - already done
# graph.add_node("tool_caller", call_tool)

# Set the entry point of the graph - already done
# graph.set_entry_point("tool_caller")

# Set the finish point of the graph to the tool_caller node
graph.set_finish_point("tool_caller")

# Compile the graph
app = graph.compile()

# Define the initial state - already done
initial_state = GraphState()

# Invoke the compiled graph
result = app.invoke(initial_state)

# Print the output
print("Final State:", result)

InvalidUpdateError: Expected dict, got <__main__.GraphState object at 0x7cd2458ce1e0>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE

**Reasoning**:
The error message `InvalidUpdateError: Expected dict, got <__main__.GraphState object at 0x...>` indicates that the state update is not in the expected dictionary format. Although the `call_tool` function is returning a dictionary, the state being passed around in the graph is still the `GraphState` object. The `GraphState` class needs to inherit from `TypedDict` to work correctly with LangGraph's state management. Also, ensure that the state class is defined correctly and used consistently. Let's redefine the `GraphState` class inheriting from `TypedDict` and then re-run the graph.



In [ ]:
from typing import TypedDict, List, Annotated
from langgraph.graph import StateGraph, END
import operator

# Define the state as a TypedDict
class GraphState(TypedDict):
    """Represents the state of our graph."""
    messages: Annotated[List[str], operator.add]
    tool_calls: Annotated[List[dict] | None, operator.add]
    next: str | None

# Create an instance of StateGraph
graph = StateGraph(GraphState)

# Define a node to handle tool calls
def call_tool(state: GraphState):
    print("Calling tool...")
    # The node should return updates to the state as a dictionary
    return {"messages": ["Tool called"], "next": "end"}

# Add the defined nodes to the StateGraph instance
graph.add_node("tool_caller", call_tool)

# Set the entry point of the graph
graph.set_entry_point("tool_caller")

# Set the finish point of the graph
graph.set_finish_point("tool_caller")

# Compile the graph
app = graph.compile()

# Define the initial state
# The initial state should also be a dictionary conforming to GraphState
initial_state: GraphState = {"messages": [], "tool_calls": None, "next": None}

# Invoke the compiled graph
result = app.invoke(initial_state)

# Print the output
print("Final State:", result)

Calling tool...
Final State: {'messages': ['Tool called'], 'tool_calls': None, 'next': 'end'}


## Summary:

### Data Analysis Key Findings

*   The `langgraph` library was successfully installed, including its dependencies.
*   Two dummy tools, `tool1` and `tool2`, were defined using the `@tool` decorator from `langchain_core.tools`.
*   A basic LangGraph `StateGraph` was defined with a single node named `tool_caller` which simulates calling a tool.
*   Initially, running the graph failed with an `InvalidUpdateError` because the `GraphState` was defined as a custom class instead of inheriting from `typing.TypedDict`, which is required for LangGraph's state management.
*   After redefining `GraphState` as a `TypedDict` and providing the initial state as a dictionary, the graph executed successfully.
*   The final state of the graph shows the message "Tool called" was added, indicating the `call_tool` node was executed.

### Insights or Next Steps

*   Future development should focus on integrating actual tool execution within the `call_tool` node and handling the results to update the graph state appropriately.
*   The graph can be expanded to include more complex logic, such as conditional routing based on tool outputs or the addition of an agent node to decide which tool to call.
